### Load libraries and external data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import colored
from colored import stylize
from data import dataloader
import datetime
import json
from neuralnet import nn
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
#import pixiedust_node #v≥0.2.5

#### Load data from Firebase.
Requires [Firebase service account credentials](https://console.firebase.google.com/project/tingle-pilot-collected-data/settings/serviceaccounts/adminsdk) in JSON format saved in `./firebase-credentials`.

In [3]:
pilot_data, notes = dataloader.load_from_firebase(
    notes=True,
    start=datetime.datetime(2018,3,6,8)
)

Data loaded from Firebase!


In [18]:
c1 = pilot_data[pilot_data.thermopile1 != False].set_index("human-readable timestamp")
c2 = pilot_data[pilot_data.thermopile1 == False].set_index("human-readable timestamp")
c2 = c2.reindex(c1.index, method="ffill")
c1[['firstCoordinator', 'coordinator1']] = c1[['coordinator', 'ontarget']]
c2[['secondCoordinator', 'coordinator2']] = c2[['coordinator', 'ontarget']]
c2 = c2[['secondCoordinator', 'step', 'coordinator2']]
c1.drop(["coordinator", "ontarget"], axis=1, inplace=True)
c = pd.concat([c1, c2], axis=1)
c["ontarget"] = c[c["coordinator1"] == c["coordinator2"]]["coordinator1"]

In [21]:
c[c.ontarget == True]

,distance,hand,pitch,question,roll,section,step,target,thermopile1,thermopile2,thermopile3,thermopile4,timestamp,firstCoordinator,coordinator1,secondCoordinator,step,coordinator2,ontarget
human-readable timestamp,,,,,,,,,,,,,,,,,,,
2018-03-06 13:31:57.048,155.0,R,227.0,Eat one of the items of food in front of you (...,63.0,B,1,food,75.4,82.3,76.6,80.9,1520343117048,Jake Son,True,Jon Clucas,1,True,True
2018-03-06 13:31:57.247,153.0,R,229.0,Eat one of the items of food in front of you (...,44.0,B,1,food,75.3,82.5,76.6,80.9,1520343117247,Jake Son,True,Jon Clucas,1,True,True
2018-03-06 13:31:57.449,139.0,R,230.0,Eat one of the items of food in front of you (...,39.0,B,1,food,75.3,82.1,77.0,80.8,1520343117449,Jake Son,True,Jon Clucas,1,True,True
2018-03-06 13:31:57.649,103.0,R,232.0,Eat one of the items of food in front of you (...,47.0,B,1,food,75.1,81.6,76.3,80.3,1520343117649,Jake Son,True,Jon Clucas,1,True,True
2018-03-06 13:32:06.847,149.0,R,232.0,Eat one of the items of food in front of you (...,80.0,B,2,food,75.8,80.6,74.9,80.0,1520343126847,Jake Son,True,Jon Clucas,2,True,True
2018-03-06 13:32:07.047,169.0,R,203.0,Eat one of the items of food in front of you (...,33.0,B,2,food,75.8,82.8,75.5,81.3,1520343127047,Jake Son,True,Jon Clucas,2,True,True
2018-03-06 13:32:07.449,123.0,R,231.0,Eat one of the items of food in front of you (...,36.0,B,2,food,75.0,83.1,76.0,81.1,1520343127449,Jake Son,True,Jon Clucas,2,True,True
2018-03-06 13:32:13.649,167.0,R,211.0,Eat one of the items of food in front of you (...,45.0,B,3,food,74.4,79.1,76.0,79.3,1520343133649,Jake Son,True,Jon Clucas,3,True,True
2018-03-06 13:32:13.847,163.0,R,251.0,Eat one of the items of food in front of you (...,79.0,B,3,food,74.3,81.9,75.9,80.4,1520343133847,Jake Son,True,Jon Clucas,3,True,True


In [ ]:
d[(~d.timestamp.isin(c.timestamp)) & (d.ontarget==True)]

In [ ]:
d = dataloader.combine_coordinators(pilot_data)

In [ ]:
participant1 = dataloader.split_participants(pilot_data)[0]

#### Load [Synaptic](http://caza.la/synaptic/)
If "Error: Cannot find module 'synaptic'", create and run these two cells:

1. ```
cd neuralnet
```

2. ```sh
!npm init -y
!npm install -s synaptic
```

In [ ]:
%%node
var lstm = require('../../tingle-pilot-study/neuralnet/lstm.js');

---
### See all targets, number of available samples and iteration blocks

In [ ]:
with open("neuralnet/targets.json", 'r') as fp:
    targets = json.load(fp)

{target: training, [all offtarget], [all onbody offtarget], [all nontraining rotation], [all offbody]}

In [ ]:
for target in list(pilot_data.target.unique()):
    ib =max(
        pilot_data.loc[
            pilot_data.target==target
        ]["step"].dropna()
    )
    print(": ".join([
        target,
        "{0} on-target samples in step {1}".format(
            str(len(pilot_data.loc[
                (pilot_data.target == target) &
                (pilot_data.ontarget)
            ])),
            "%.0f" % ib
        )
    ]))

---
### Extract training and testing data
Define targets of interest and corresponding offtargets

with open(
    'data/targets.json',
    'r'
) as fp:
    targets = json.load(
        fp
    )[0]

Set parameters for nn:

In [ ]:
input_signals = [
    "distance",
    "thermopile1",
    "thermopile2",
    "thermopile3",
    "thermopile4"
]
n_samples = [300, 250, 200, 150, 100, 50]
steps = list(range(1, 48))

Get training inputs and outputs, inputs that should evaluate ~true and inputs that should evaluate ~false

In [ ]:
targ = "eyebrow"
data = nn.define_trainer_data(
    pilot_data,
    {
        "target": [targets[targ]],
        "offtarget": targets[targ][1]
    },
    input_signals,
    steps,
    n_samples=n_samples[0]
)

Preview all inputs and training outputs

In [ ]:
data

**Note**: These data take some time to copy across environments. Give the notebook some time between running cells across Python and JavaScript.

---
### Train and test

In [ ]:
%%node
var networks = {};
for n_sample in n_samples:
    for (var target in data) {
      networks[target] = lstm.train_lstm([5,5,2,1], data[target]["train"], 0.06, 0.06, 3000);
    }

In [ ]:
%%node
var test_outputs = {};
for (var target in data) {
    test_outputs[target] = {"true":[],"false":[]};
    for(var iteration=0; iteration < data[target]["test_true"].length; iteration++){
      test_outputs[target]["true"].push(networks[target].activate(data[target]["test_true"][iteration]));
      }
    for(var iteration=0; iteration < data[target]["test_false"].length; iteration++){
      test_outputs[target]["false"].push(networks[target].activate(data[target]["test_false"][iteration]));
    }
}

In [ ]:
test_outputs

---
### See outputs

In [ ]:
def calc_confusion(negative, positive):
    """
    Function to calculate a confusion matrix
    
    Parameters
    ----------
    negative: list of floats
        outputs of neural nets with true negative inputs
        
    positive: list of floats
        outputs of neural nets with true positive inputs
        
    Returns
    -------
    confusion: matrix of floats
        tn, fp, fn, tp
        see http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
    """
    ytrue = [
        *[
            0 for output in negative
        ],
        *[
            1 for output in positive
        ]
    ]
    ypredicted = [
        *[
            int(round(o)) for o in negative
        ],
        *[
            int(round(o)) for o in positive
        ]
    ]
    return(confusion_matrix(ytrue, ypredicted))

If the training is adequate x ≈ 0 ∀ x in the following:

In [ ]:
f = {target: [
        outputs for outputs in test_outputs[target]['false']
] for target in targets}
f

If the training is adequate x ≈ 1 ∀ x in the following:

In [ ]:
t = {target: [
        outputs for outputs in test_outputs[target]['true']
] for target in targets}
t

Finally, if training is adequate, f ≪ t:

In [ ]:
f_mean = {
    target: np.mean(f[target]) for target in targets if target in f and len(f[target])
}
t_mean = {
    target: np.mean(t[target]) for target in targets if target in t and len(t[target])
}
for target in t_mean:
    print(target)
    print(
        "f = {0}\nt = {1}\n{0} ≪ {1} ?\n".format(
            str(f_mean[target]),
            str(t_mean[target])
        ) if f_mean[target] < t_mean[target] else "f = {0}\nt = {1}\n{2}".format(
            str(f_mean[target]),
            str(t_mean[target]),
            stylize(
                "Nope. f > t\n",
                colored.fg("red")
            )
        )
    )
for target in t_mean:
    print("{0}: f = {1:.4f} < t = {2:.4f}".format(
        target,
        f_mean[target],
        t_mean[target]
    ))